# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
#Load the csv exported from Part 1
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hilo,19.73,-155.09,84.2,58,1,11.41,US,1607303597
1,1,Glendive,47.11,-104.71,36.0,66,1,9.17,US,1607304253
2,2,Puerto Ayora,-0.74,-90.35,72.0,91,72,3.00,EC,1607303963
3,3,Norman Wells,65.28,-126.83,1.4,84,90,3.36,CA,1607304161
4,4,Coquimbo,-29.95,-71.34,57.2,82,75,8.05,CL,1607303908


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]

In [30]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

![heat_map.png](../Images/heat_map.png)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
#Narrow down data for ideal weather condition >=75 and <80, 0 cloudiness and wind <10
#Identify rows with needed data using the loc method
weather_new = cities_df.loc[(cities_df["Max Temp(F)"] >=75) &
                           (cities_df["Wind Speed"] < 10) &
                           (cities_df["Cloudiness"] <= 0)]


In [32]:
#The output has 9 rows and we drop the row that does not meet the all the criterias
drop_value = weather_new.drop(weather_new.index[2])

#We reset the index to later create a dataframe
new_weather_index = drop_value.reset_index(drop=True)
del new_weather_index['Unnamed: 0']


In [33]:
#New dataframe of 8 cities fitting the criteria 
weather_df = pd.DataFrame(new_weather_index)

#Add a new column for hotel name
weather_df['Hotel Name'] = ""
weather_df.head(10)

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Acajutla,13.59,-89.83,78.80,57,0,2.01,SV,1607304337,
1,Takoradze,4.88,-1.76,78.89,83,0,7.18,GH,1607304411,
2,Itupiranga,-5.13,-49.33,78.80,100,0,2.19,BR,1607304514,
3,Cayenne,4.93,-52.33,77.00,100,0,8.23,GF,1607304226,
4,Belmonte,-15.86,-38.88,77.00,78,0,6.93,BR,1607304172,
5,Niamey,13.51,2.11,78.80,27,0,6.15,NE,1607304680,
6,Uruçuí,-7.23,-44.56,78.44,69,0,1.34,BR,1607304770,
7,Ormara,25.21,64.64,75.47,71,0,4.29,PK,1607304854,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
# geocoordinates
target_search = "hotel"
target_radius = 5000

# set up a parameters dictionary "location": f"{lat},{lng}",
params = { "keyword": target_search,
            "radius": target_radius,
            "key": g_key
}


for index, row in weather_df.iterrows():
    
    # get lat and lng from df
    lat = row['Lat']
    lng= row['Lng']
    
    #Change location each iteration while leaving params the same
    params["location"] = f"{lat},{lng}"
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        weather_df.loc[index, 'Hotel Name'] = results [0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
                                         

In [35]:
hotel_df = weather_df 
hotel_df.head()

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Acajutla,13.59,-89.83,78.80,57,0,2.01,SV,1607304337,Villavela Hotel Boutique
1,Takoradze,4.88,-1.76,78.89,83,0,7.18,GH,1607304411,Milgwen Hotel
2,Itupiranga,-5.13,-49.33,78.80,100,0,2.19,BR,1607304514,Hotel Beira Rio
3,Cayenne,4.93,-52.33,77.00,100,0,8.23,GF,1607304226,Royal Amazonia
4,Belmonte,-15.86,-38.88,77.00,78,0,6.93,BR,1607304172,Pousada Caçari


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

![hotel_map.png](../Images/hotel_map.png)